In [1]:
import dihz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The dihz started successfully.


## 斯特潘—玻尔兹曼常数
$\sigma=5.66956*10^{-12} J cm^{-2}K^{-4}s^{-1}$
## 恒星质量与半径，有效温度线性拟合系数
$R = a*M+b$    
$T = a*M+b$

In [2]:
#Determine habitability
def determine_hb(a,b,c,d):
    
    if a>c and b<d:
        return 1 #liveable
    elif a>d or b<c:
        return 2 #unliveable
    else:
        return 0 #part livable

In [3]:
#data loading
file = 'dataset.csv'
df = pd.read_csv(file, sep=',' )
df

,m1,m2,r1,r2,t1,t2,e_b,a_p,inc,status,min_dis,max_dis
0,0.51,0.49,0.5424,0.5176,4126.5114,4068.9286,0.55,0.59,0.523599,-1,-1.000000,-1.000000
1,0.63,0.37,0.6912,0.3688,4472.0082,3723.4318,0.63,1.16,2.967060,-1,-1.000000,-1.000000
2,0.67,0.33,0.7408,0.3192,4587.1738,3608.2662,0.20,0.93,0.349066,1,0.846892,1.008940
3,0.69,0.31,0.7656,0.2944,4644.7566,3550.6834,0.34,1.07,0.698132,1,0.973318,1.155326
4,0.71,0.29,0.7904,0.2696,4702.3394,3493.1006,0.25,1.16,2.268928,1,1.069090,1.248977
...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0.64,0.36,0.7036,0.3564,4500.7996,3694.6404,0.56,0.61,0.000000,-1,-1.000000,-1.000000
599996,0.80,0.20,0.9020,0.1580,4961.4620,3233.9780,0.50,0.75,0.174533,1,0.623241,0.869252
599997,0.57,0.43,0.6168,0.4432,4299.2598,3896.1802,0.76,1.38,0.872665,-1,-1.000000,-1.000000
599998,0.78,0.22,0.8772,0.1828,4903.8792,3291.5608,0.25,0.90,0.000000,1,0.798659,0.989467


In [4]:
#Screening stable sample
stable=df[df['status']==1]
L1=stable.apply(lambda x: x['r1']**(2) * (x['t1']/5800.)**(4), axis=1)
L2=stable.apply(lambda x: x['r2']**(2) * (x['t2']/5800.)**(4), axis=1)
stable.insert(6,'L2',L2)
stable.insert(6,'L1',L1)
stable.head(5)

,m1,m2,r1,r2,t1,t2,L1,L2,e_b,a_p,inc,status,min_dis,max_dis
2,0.67,0.33,0.7408,0.3192,4587.1738,3608.2662,0.214719,0.015262,0.20,0.93,0.349066,1,0.846892,1.008940
3,0.69,0.31,0.7656,0.2944,4644.7566,3550.6834,0.241070,0.012173,0.34,1.07,0.698132,1,0.973318,1.155326
4,0.71,0.29,0.7904,0.2696,4702.3394,3493.1006,0.269922,0.009563,0.25,1.16,2.268928,1,1.069090,1.248977
5,0.63,0.37,0.6912,0.3688,4472.0082,3723.4318,0.168852,0.023102,0.17,0.93,1.047198,1,0.854561,1.002811
7,0.53,0.47,0.5672,0.4928,4184.0942,4011.3458,0.087130,0.055564,0.06,0.93,2.792527,1,0.870399,0.985991


In [5]:
#Calculating the habitable zone
M1 = stable[['m1', 'r1', 't1','L1','e_b']].to_dict('list')
M2 = stable[['m2', 'r2', 't2', 'L2']].to_dict('list')
ab = 0.1
num = len(stable['m1'])
phzi = np.zeros(num)
phzo = np.zeros(num)
ahzi = np.zeros(num)
ahzo = np.zeros(num)
for i in range(num):
    [PHZI,PHZO]=dihz.circumbinary.PHZ(M1['L1'][i],M1['t1'][i],M1['m1'][i],M2['L2'][i],M2['t2'][i],M2['m2'][i],ab,M1['e_b'][i])
    [AHZI,AHZO]=dihz.circumbinary.AHZ(M1['L1'][i],M1['t1'][i],M1['m1'][i],M2['L2'][i],M2['t2'][i],M2['m2'][i],ab,M1['e_b'][i])
    phzi[i] = PHZI 
    phzo[i] = PHZO
    ahzi[i] = AHZI
    ahzo[i] = AHZO
n = len(stable.loc[2,:])
stable.insert(n, 'phzo', phzo)
stable.insert(n, 'phzi', phzi)
stable.insert(n, 'ahzo', ahzo)
stable.insert(n, 'ahzi', ahzi)

In [6]:
#reset index
stable = stable.reset_index(drop=True)
stable['phb'] = stable.apply(lambda x: determine_hb(x.min_dis, x.max_dis, x.phzi, x.phzo), axis=1)
ahb = stable.apply(lambda x: determine_hb(x.min_dis, x.max_dis, x.ahzi, x.ahzo), axis=1)
stable.insert(16,'ahb',ahb)
stable.head(10)

,m1,m2,r1,r2,t1,t2,L1,L2,e_b,a_p,inc,status,min_dis,max_dis,ahzi,ahzo,ahb,phzi,phzo,phb
0,0.67,0.33,0.7408,0.3192,4587.1738,3608.2662,0.214719,0.015262,0.20,0.93,0.349066,1,0.846892,1.008940,0.484926,0.896208,0,0.579651,0.826472,2
1,0.69,0.31,0.7656,0.2944,4644.7566,3550.6834,0.241070,0.012173,0.34,1.07,0.698132,1,0.973318,1.155326,0.507601,0.935084,2,0.627018,0.846665,2
2,0.71,0.29,0.7904,0.2696,4702.3394,3493.1006,0.269922,0.009563,0.25,1.16,2.268928,1,1.069090,1.248977,0.531895,0.976763,2,0.634504,0.896201,2
3,0.63,0.37,0.6912,0.3688,4472.0082,3723.4318,0.168852,0.023102,0.17,0.93,1.047198,1,0.854561,1.002811,0.445112,0.828037,2,0.532010,0.770909,2
4,0.53,0.47,0.5672,0.4928,4184.0942,4011.3458,0.087130,0.055564,0.06,0.93,2.792527,1,0.870399,0.985991,0.386943,0.728782,2,0.426733,0.723492,2
5,0.75,0.25,0.8400,0.2200,4817.5050,3377.9350,0.335842,0.005569,0.23,0.81,3.141593,1,0.714367,0.901758,0.584799,1.067556,1,0.679968,0.988426,1
6,0.74,0.26,0.8276,0.2324,4788.7136,3406.7264,0.318276,0.006428,0.03,0.66,2.094395,1,0.580538,0.737249,0.571071,1.043996,1,0.628737,0.994400,0
7,0.53,0.47,0.5672,0.4928,4184.0942,4011.3458,0.087130,0.055564,0.22,1.39,1.570796,1,1.324158,1.454602,0.386943,0.728782,2,0.439385,0.722568,2
8,0.61,0.39,0.6664,0.3936,4414.4254,3781.0146,0.149023,0.027979,0.00,1.21,2.792527,1,1.146274,1.270925,0.428348,0.799385,2,0.487655,0.762716,2
9,0.59,0.41,0.6416,0.4184,4356.8426,3838.5974,0.131070,0.033586,0.36,0.57,1.396263,1,0.483044,0.649756,0.413957,0.774815,1,0.516884,0.723243,0


In [ ]:
stable.to_csv('./habitable_dataset.csv', index=False)